# Setup

In [3]:
!go mod init k8swithgo

go: creating new go.mod: module k8swithgo


In [42]:
!go get "k8s.io/client-go/kubernetes@latest"

# client-go

## kubernetes

core clientset for API access

In [48]:
import (
    "context"
    "fmt"
    "os"
    "path/filepath"
    "k8s.io/client-go/tools/clientcmd"
    "k8s.io/client-go/kubernetes"
    metav1 "k8s.io/apimachinery/pkg/apis/meta/v1"
    appsv1 "k8s.io/api/apps/v1"
    corev1 "k8s.io/api/core/v1"
)

### Authentication

Handles configuration-based auth (`~/.kube/config`) for out-of-cluster, i.e. how `kubectl` works
In-cluster auth relies on credentials being injected into a running pod from within the cluster

```go
config, err := rest.InClusterConfig()
```

In [3]:
func getKubeClient() *kubernetes.Clientset {
    home, _ := os.UserHomeDir()
    kubeConfigPath := filepath.Join(home, ".kube", "config")

    config, err := clientcmd.BuildConfigFromFlags("", kubeConfigPath)
    if err != nil {
        panic(err)
    }
    client := kubernetes.NewForConfigOrDie(config)
    return client
}

### List Resources

In [10]:
%%
client := getKubeClient()
pods, _ := client.CoreV1().Pods("insights-agent").List(context.TODO(), metav1.ListOptions{})
for _, pod := range pods.Items {
	fmt.Println(pod)
}

{{ } {insights-agent-insights-right-sizer-controller-544c6bd8f6-dtf2p insights-agent-insights-right-sizer-controller-544c6bd8f6- insights-agent  a278ef2f-81ff-40e4-aae5-b38fc4a9bf44 227795 0 2024-02-16 10:34:36 -0600 CST <nil> <nil> map[app.kubernetes.io/component:controller app.kubernetes.io/instance:insights-agent app.kubernetes.io/name:insights-right-sizer pod-template-hash:544c6bd8f6] map[] [{apps/v1 ReplicaSet insights-agent-insights-right-sizer-controller-544c6bd8f6 23c6e32e-34d8-488d-8fc2-a8af3c0770bb 0x1400068f157 0x1400068f158}] [] [{kube-controller-manager Update v1 2024-02-16 10:34:36 -0600 CST FieldsV1 {"f:metadata":{"f:generateName":{},"f:labels":{".":{},"f:app.kubernetes.io/component":{},"f:app.kubernetes.io/instance":{},"f:app.kubernetes.io/name":{},"f:pod-template-hash":{}},"f:ownerReferences":{".":{},"k:{\"uid\":\"23c6e32e-34d8-488d-8fc2-a8af3c0770bb\"}":{}}},"f:spec":{"f:containers":{"k:{\"name\":\"insights-right-sizer\"}":{".":{},"f:command":{},"f:image":{},"f:imageP

## Labels

In [27]:
func labelDeployment(client *kubernetes.Clientset, namespace string, deployment string) error {
	ctx := context.TODO()
	deploy, err := client.AppsV1().Deployments(namespace).Get(ctx, deployment, metav1.GetOptions{})
	if err != nil {
		panic(err)
	}
	fmt.Println(deploy.Labels)
	deployLabels := deploy.Labels
	if deployLabels == nil {
		deployLabels = make(map[string]string)
	}
	deployLabels["fairwinds.com/demo"] = "true"
	deploy.Labels = deployLabels
	deploy, err = client.AppsV1().Deployments(namespace).Update(ctx, deploy, metav1.UpdateOptions{})
	return err
}

In [32]:
%%
client := getKubeClient()
err := labelDeployment(client, "default", "nginx-deployment")
if err != nil {
	panic(err)
}

ctx := context.TODO()
deploy, err := client.AppsV1().Deployments("default").List(ctx, metav1.ListOptions{
	LabelSelector: "fairwinds.com/demo=true",
})
if err != nil {
	panic(err)
}

fmt.Println(deploy)

map[fairwinds.com/demo:true]
&DeploymentList{ListMeta:{ 232602  <nil>},Items:[]Deployment{Deployment{ObjectMeta:{nginx-deployment  default  eb65bde0-9624-480e-8de0-5eaa8ca95e61 232300 2 2024-02-12 13:34:52 -0600 CST <nil> <nil> map[fairwinds.com/demo:true] map[deployment.kubernetes.io/revision:2 kubectl.kubernetes.io/last-applied-configuration:{"apiVersion":"apps/v1","kind":"Deployment","metadata":{"annotations":{},"name":"nginx-deployment","namespace":"default"},"spec":{"replicas":2,"selector":{"matchLabels":{"app":"nginx"}},"template":{"metadata":{"labels":{"app":"nginx"}},"spec":{"containers":[{"image":"nginx:1.14.2","name":"nginx","ports":[{"containerPort":80}]}]}}}}
] [] [] [{kubectl-client-side-apply Update apps/v1 2024-02-12 13:34:52 -0600 CST FieldsV1 {"f:metadata":{"f:annotations":{".":{},"f:kubectl.kubernetes.io/last-applied-configuration":{}}},"f:spec":{"f:progressDeadlineSeconds":{},"f:replicas":{},"f:revisionHistoryLimit":{},"f:selector":{},"f:strategy":{"f:rollingUpdate":

## Create a Pod!

In [59]:
func createPod(client *kubernetes.Clientset) error {
	ctx := context.TODO()

	pod := &corev1.Pod{
        ObjectMeta: metav1.ObjectMeta{
            Name:      "test-pod",
            Namespace: "default",
            Labels: map[string]string{
                "app": "test-pod",
            },
        },
        Spec: corev1.PodSpec{
            Containers: []corev1.Container{
                {
                    Name:            "busybox",
                    Image:           "busybox",
                    ImagePullPolicy: corev1.PullIfNotPresent,
                    Command: []string{
                        "sleep",
                        "3600",
                    },
                },
            },
        },
    }

	if _, err := client.CoreV1().Pods("default").Create(ctx, pod, metav1.CreateOptions{}); err != nil {
		return err
	} else {
		return nil
	}
}

In [61]:
%%
client := getKubeClient()
err := createPod(client)
if err != nil {
	panic(err)
}

# Dynamic

Can perform generic operations on arbitrary K8S API objects

* `FilteredDynamicSharedInformerFactory`: watch `Unstructured` objects with `AddFunc/UpdateFunc/DeleteFunc` and handle with event handlers
* shared caching

examples of dynamic client in https://github.com/FairwindsOps/insights-plugins/tree/main/realtime-reporter/pkg

![alt text](media/informers.png "Informers")

## Informer Interface

```go
type Informer interface {
	AddEventHandler(handler handlers.EventHandler)
	HasSynced() bool
	Start(ch <-chan struct{})
}
```

```go
// AddEventHandler adds the handler to each namespaced informer
func (i *multiResourceInformer) AddEventHandler(handler handlers.EventHandler) {
	for _, ki := range i.resourceToInformer {
		for kind, informer := range ki {
			informer.AddEventHandler(handler(kind))
		}
	}
}

// HasSynced checks if each namespaced informer has synced
func (i *multiResourceInformer) HasSynced() bool {
	for _, ki := range i.resourceToInformer {
		for _, informer := range ki {
			if ok := informer.HasSynced(); !ok {
				return ok
			}
		}
	}

	return true
}

func (i *multiResourceInformer) Start(stopCh <-chan struct{}) {
	for _, informer := range i.informerFactory {
		informer.Start(stopCh)
	}
}
```

# Controllers & Operators

## Controllers

Built-in, monitor core k8s objects

Examples:

* `apps/v1/Deployment`
* `core/v1/Pod`
* `batch/v1/Job` `batch/v1/CronJob`


## Operators

Controllers that operate on custom resources (CRDs). Extends capabilities of Kubernetes

Examples:

* `acme.cert-manager.io/Challenge` `acme.cert-manager.io/Order` `acme.cert-manager.io/Certificate`
* `argoproj.io/Workflow` `argoproj.io/Application` `argoproj.io/Rollout`
* `keda.sh/ScaledOject`
* `monitoring.grafana.com/MetricsInstance`

## Frameworks

* [kubebuilder](https://github.com/kubernetes-sigs/kubebuilder)
* [operator framework](https://operatorframework.io/)